# Using Data Science to find the best place to live in Sydney
*The Battle of Neighborhoods - IBM Data Science Capstone Project*

## Introduction
Sydney is a fast growing city with over 600 suburbs and has been high up on the list for most liveable cities in the world for several years. It is no suprise that many people dream of visiting or starting a life in Sydney. 

In 2019 I had the privledge of going on exchange to Sydney for one semester, and I can safely say that it is a great place to be! However, it is also one of the most expensive cities in the world, and with so many suburbs it is hard to know which places that one should consider when choosing where to live. This is where the idea for this project started. 

What if we could use Data Science to recommend which places would be of interest for someone planning on moving to Sydney, or even just choosing a place to stay temporarily?

In this project, the goal is to make suggestions based on three main factors*:
* Proximity to recommended venues.
* Living expenses.
* Safety.

___

\**DISCLAIMER: I do not encourage anyone to make their decisions based on the findings in this report alone. There are many more factors to consider than are captured in the scope of this project.* 

## Data
The data for this project will be gathered from various sources using API calls, web scraping and publicly available datasets. 

### Proximity to recommended venues
Foursquare API will be used for gathering information about the most popular venue types in each suburb. This will be done to recommend the most suitable places to live based on a stakeholder's venues of interest.

Australian post codes with coordinates https://www.matthewproctor.com/australian_postcodes. Australia Post charges for using their dataset. Instead, a regularly updated, community maintained dataset will be used for this project. Sydney suburb names with coordinates will be extracted from this dataset and then used with the Foursquare API to list venues in each suburb for recommendation.

### Living expenses
The median house value in each suburb will be web scraped from http://house.speakingsame.com/suburbtop.php?sta=nsw&cat=HomePrice&name=. Geojson data will be gathered from https://data.gov.au/dataset/ds-dga-91e70237-d9d1-4719-a82f-e71b811154c6/details which contains suburb names and boundary coordinates. This will be used for visualizing suburbs with boundaries. The geojson data will then be used together with the median house values to get an overview of which areas are less or more expensive to live in Sydney. 

### Safety
Crime reports in CSV format from 2019 for every Local Government Area (LGA) in Sydney was retrieved from the NSW Bureau of Crime Statistics and Research https://www.bocsar.nsw.gov.au/Pages/bocsar_crime_stats/bocsar_latest_quarterly_and_annual_reports.aspx. This will be done so that a stakeholder can find the safest regions in Sydney. LGA geojson was gathered from https://data.gov.au/dataset/ds-dga-f6a00643-1842-48cd-9c2f-df23a3a1dc1e/details to use for visualizing areas in a choropleth map.



## Methodology

We start by importing necessary libraries and select the coordinates for Sydney that the map visualizations will use.

In [ ]:
import numpy as np
import pandas as pd
import requests
# If folium is not installed
#!conda install -c conda-forge folium --yes
import folium
import json

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# display all dataframe rows
#pd.set_option('display.max_rows', None)

sydney_coords = [-33.8688, 151.2093] 

### Recommended Venues

#### Dataframe for venues
First we'll need to create a dataframe with all Sydney suburbs and their corresponding coordinates. I use a community maintained dataframe where I have manually changed a few inaccurate coordinates for some suburbs. This dataframe contains all australian postcodes, so I merge it with another dataframe only containing Sydney postcodes and suburb names. I select the intersection of these two dataframes by using inner join, in order to get a dataframe with Sydney suburbs only.

In [ ]:
df_coord = pd.read_csv('https://github.com/saammirghorbani/australianpostcodes/raw/master/australian_postcodes.csv', usecols=['postcode', 'locality', 'long', 'lat'])
df_subs = pd.read_csv('sydney_suburbs_postcodes.csv')


# Name must be uppercase to match df_cords 'locality' column 
df_subs['locality'] = df_subs['locality'].str.upper()
df_vnu = pd.merge(df_coord, df_subs, how='inner', on=['locality', 'postcode'])
# Give better names to columns

df_vnu.rename(columns={'locality':'suburb'}, inplace=True)

df_vnu.dropna(inplace=True)

print(df_vnu.shape)
df_vnu.head()

A problem with this dataframe is that was found when clustering neighborhoods was that there are many neighboring suburbs that share the same coordinates. This means that we shouldn't use these coordinates to look for venues, as they would return the same venues for all the neighboring suburbs, and in the end it would incorrectly cluster them in the same group even though they most likely have different compositions of venues. 

As I mentioned, I have manually updated the coordinates on some of these suburbs, but since it would take a very long time to go through all we will simply keep a subset of suburbs that don't share coordinates for our clustering on venues.

In [ ]:
# Drop suburbs that have the same coordinates, keep the first occurence.
df_vnu.drop_duplicates(subset=['long', 'lat'], keep='first', inplace=True)
print(df_vnu.shape)

#### Foursquare API
In the next stage I want to get the most popular venues for each suburb. For this I make calls to Foursquare API. 

In [ ]:
CLIENT_ID = '5XXK41DFPB522NKMS0JSWEE0EQ0PC0JKPCZ14IYOFI3EWURX'#'CRALOD0T0CQVYXKE1VXXA0BMCD1HEIBU3VGFIFJBVRPP1W0J'#'2LPZZ10LCNP1RNKTFGKGUWRSIS5MXYSFU1F53VVZ3XASYOXO'#'5XXK41DFPB522NKMS0JSWEE0EQ0PC0JKPCZ14IYOFI3EWURX'  # your Foursquare ID
CLIENT_SECRET = 'THPV0ULRUMHAAV3EOFXWTFPVGCCBPUTJZNOZQYFNPHBFZLYD'#'X4JPJATKV3LOVTPOW5T3OR33MO0XZFPXN03KC5W2EJ15FHGV'#'0CQ4R2URVJDAATZDYKU3OSWUSFTNFE2N45MMKGMWRRQ4JPMB'#  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Let's define a method that given a suburb name with coordinates returns the top 100 veunes within a radius of 500 m.

In [ ]:
def get_venues(suburb_names, longitudes, latitudes):
    LIMIT = 100
    radius = 500 
    venues_list = []
    for name, long, lat in zip(suburb_names, longitudes, latitudes): 
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']

         # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['categories'][0]['name']) for v in results])
        print(name)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue Category']
    return(nearby_venues)
    


Now, let's get the top 100 venues for every suburb in our dataframe. This will take some time so I have created a CSV file containing the same data.

In [ ]:
# Comment out if you want to make API calls to Foursquare and save to file
#sydney_venues = get_venues(suburb_names=df_vnu['suburb'], longitudes=df_vnu['long'], latitudes=df_vnu['lat'])
#sydney_venues.to_csv('sydney_venues.csv',index=False)
sydney_venues = pd.read_csv('sydney_venues.csv')
sydney_venues.head()

#### Data Wrangling

Let's prepare the data for K-Means clustering by encoding categorical variables (veunes) into a series of zeroes and ones. 

In [ ]:
sydney_enc = pd.get_dummies(sydney_venues[['Venue Category']], prefix="", prefix_sep="")

# add suburb column back to dataframe
sydney_enc['Suburb'] = sydney_venues['Suburb'] 

# move suburb column to the first column
sydney_enc = sydney_enc[['Suburb'] + list(sydney_enc.columns[:-1])]

sydney_enc.head()

We only want one entry per suburb, so we group the dataframe by suburb and use the mean to get a normalized value for the occurence of each venue category in that suburb.

In [ ]:
sydney_grouped = sydney_enc.groupby('Suburb').mean().reset_index()
sydney_grouped.head()

The above dataframe will be ideal to use for K-means clustering of suburbs by similar venue categories. But to make sense of the patterns that the clusters are trying to capture we need another representation of this dataframe where we can see the top most common venues for each suburb. 

In [ ]:
#Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
suburbs_venues_sorted = pd.DataFrame(columns=columns)
suburbs_venues_sorted['Suburb'] = sydney_grouped['Suburb']

for ind in np.arange(sydney_grouped.shape[0]):
    suburbs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sydney_grouped.iloc[ind, :], num_top_venues)

suburbs_venues_sorted.head()

#### K-Means Clustering
We can now run the K-means clustering algorithm on the data with different values for K to see which one would be most suitable for our model. We use the sum of square distances from each point to its assigned center as a measure of how well the model is clustering the data. We get this as a score from the model's inertia and plot it for every value of K.

In [ ]:
# for reproducing results
rnd_state = 1  
distortions = []
features = sydney_grouped.drop('Suburb', 1)
kmeans_models = []
K = range(1, 11)
for k in K:
    kmeans_models.append(KMeans(n_clusters=k, random_state=rnd_state).fit(features))
    distortions.append(kmeans_models[k-1].inertia_)
    
plt.figure(figsize=(16, 8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.show()

Using the elbow method to visually determine the optimal value for K, we find that the inflection point happens where K = 3. 

#### Final model
We pick the model with K = 3 and merge the labels with the dataframe that shows the top venues for each suburb. 

In [ ]:
kclusters = 3
kmeans2 = kmeans_models[kclusters-1]
sydney_grouped2 = sydney_grouped.copy()
sydney_grouped2['labels'] = kmeans2.labels_
df_final=suburbs_venues_sorted.join(sydney_grouped2.set_index('Suburb'), on=['Suburb'])
df_final.drop(df_final.iloc[:, num_top_venues+1:-1], inplace=True, axis=1)
df_final = df_final.join(df_vnu.set_index('suburb'), on=['Suburb'])
df_final.head()

### House values
I created a table over suburbs with their median house values by webscraping. Then I removed dollar signs and commas so that the value is displayed as an integer.   

In [ ]:
df_house = pd.read_csv('sydney_houses_median_value.csv')
# uppercase suburb names to match geojson entries
df_house['suburb'] = df_house['suburb'].str.upper()
df_house.drop(columns=['rank'], inplace=True)
# format values for calculation
df_house = df_house.replace(r'\$','', regex=True) 
df_house = df_house.replace(r',','', regex=True)
# make sure the values are of type int
df_house['median_value_aud']=df_house['median_value_aud'].astype('int')
df_house.head()

#### Exploratory data analysis
Lets take a look at the distribution of house values by plotting them in a boxplot

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Median house price')
ax1.set_ylabel('value')
ax1.boxplot(df_house['median_value_aud'])
ax1.plot()

We see that there is an extreme outlier. Houses located in the Point Piper neighborhood are significantly more expensive (around \\$24,000,000 AUD) than houses in other neighborhoods, so let's remove it in order to better compare prices in a choropleth map. 

In [ ]:
df_house.drop(df_house.index[df_house['suburb']=='POINT PIPER'], inplace=True)

fig1, ax1 = plt.subplots()
ax1.set_title('Median house price')
ax1.set_ylabel('value')
ax1.boxplot(df_house['median_value_aud'])
ax1.plot()

### Crimes

In [ ]:
df_crime = pd.read_csv('nsw_lga_crime_trends.csv', usecols=['Local Government Area', 'Offence type','Rate per 100,000 population Jan - Dec 2019'])

df_crime.rename(columns={'Local Government Area':'region','Offence type':'type','Rate per 100,000 population Jan - Dec 2019':'crimes_2019'}, inplace=True)

df_crime['region'] = df_crime['region'].str.upper()

df_crime.dropna(inplace=True)



This dataframe includes crime data for all regions in the state of NSW, so I only select the Sydney regions.

In [ ]:
greater_sydney_regions = ['BAYSIDE', 'BLACKTOWN','BURWOOD', 'CAMDEN', 'CAMPBELLTOWN', 'CANADA BAY', 'CANTERBURY-BANKSTOWN','CUMBERLAND'
                  ,'FAIRFIELD','GEORGES RIVER','HORNSBY', 'HUNTERS HILL', 'INNER WEST','KU-RING-GAI','LANE COVE',
                  'LIVERPOOL', 'MOSMAN', 'NORTH SYDNEY', 'NORTHERN BEACHES', 'PARRAMATTA', 'PENRITH', 'RANDWICK',
                  'RYDE', 'STRATHFIELD', 'SUTHERLAND SHIRE', 'SYDNEY', 'THE HILLS SHIRE', 'WAVERLEY', 'WILLOUGHBY', 'WOOLLAHRA']
df_crime['region'] = df_crime[df_crime['region'].isin(greater_sydney_regions)]

Next, I add up all the crime rates per 100,000 population for each region and divide by 100,000 to get the crime rate as a probability. 

In [ ]:
# sum all crime rates in a region and group by region
df_crime['crimes_2019'].replace(to_replace='nc', value=0, inplace=True)
df_crime['crimes_2019'] = df_crime['crimes_2019'].astype(float)
# get crime rate
df_crime = df_crime.groupby(['region']).sum()/100000
df_crime= df_crime.sort_values(by='crimes_2019',ascending=False)
df_crime.reset_index(inplace=True)
df_crime.head()

## Results

#### Cluster composition
To categorize the clusters we can look at the most common venues in them. We print the 1st, 2nd and 3rd most common venues in order of frequency found among the suburbs in the cluster, using the dataframe we created before. 

In [ ]:
required_column_indices = [1,2,3]
required_column = [list(df_final.columns.values)[i] for i in required_column_indices]

In [ ]:
df_final_series_0 = df_final.loc[df_final['labels'] == 0, df_final.columns[0:6]]
print("Neighborhoods in cluster:", df_final_series_0['Suburb'].size)
for col in required_column:
    print('------------------------------------------')
    print(df_final_series_0[col].value_counts(ascending = False)[:5])

The most prominent venues in cluster 0 are yoga studios, fast food restaurants, and fish & chips shops.

In [ ]:
df_final_series_1 = df_final.loc[df_final['labels'] == 1, df_final.columns[0:6]]
print("Neighborhoods in cluster:", df_final_series_1['Suburb'].size)
for col in required_column:
    print('------------------------------------------')
    print(df_final_series_1[col].value_counts(ascending = False)[:5])

The most prominent venues in cluster 1 are cafés, pubs and pizza places.

In [ ]:
df_final_series_2 = df_final.loc[df_final['labels'] == 2, df_final.columns[0:6]]
print("Neighborhoods in cluster:", df_final_series_2['Suburb'].size)
for col in required_column:
    print('------------------------------------------')
    print(df_final_series_2[col].value_counts(ascending = False)[:5])

The most prominent venues in cluster 2 are parks, yoga studios and egyptian restaurants.

#### Visualizing clusters on a map

Let's plot the neighborhoods with their labels on a map to get a geographical overview.

In [ ]:
# create map
map_clusters = folium.Map(location=sydney_coords, zoom_start=10)
# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['lat'],df_final['long'], df_final['Suburb'], df_final['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

As we can see, in the case of cluster 0 and cluster 1, there seems to be some sort of connection between the geographical location of the suburbs and their labels. Cluster 1 is mostly concentrated in the city center, while cluster 0 is further out and around the edges.

#### Choropleth map over house prices

Using the dataframe from above containing median house values together with geojson data over Sydney suburbs we can visualize the values in a choropleth map.

In [ ]:
with open('sydney_suburbs_geo.json') as json_data:
    data = json.load(json_data)
    
threshold_scale = np.linspace(df_house['median_value_aud'].min(),
                              df_house['median_value_aud'].max(),
                              7, dtype=np.dtype('float64'))
threshold_scale = threshold_scale.tolist() 
# make sure that the last value of the list is greater than the maximum value
threshold_scale[-1] = threshold_scale[-1] + 1 

# create a world map
world_map = folium.Map(location=sydney_coords, zoom_start=11, tiles='OpenStreetMap')

# generate choropleth map
folium.Choropleth(
    geo_data=data,
    data=df_house,
    columns=['suburb','median_value_aud'],
    key_on='feature.properties.nsw_loca_2',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Median House value',
    threshold_scale=threshold_scale,
    reset=True
).add_to(world_map)

folium.LayerControl().add_to(world_map)
# display map
world_map

After Point Piper, we see that Watsons Bay and Elizabeth Bay (in Sydney Harbour) are the most expensive neighborhoods with most houses starting at \\$7,453,600 AUD

A suburb which stands out is Duffys Forest (north). Being so far away from the city you'd expect it to have a similar house value as surrounding suburbs. The reason why that's not the case is that this suburb has a small population but several big mansions.  

#### Choropleth map over crime rates

Using the crime rate dataframe from above we load the geojson data for NSW regions and display the crime rates on a choropleth map.

In [ ]:
with open('nsw_lga_geo.json') as json_data:
    data2 = json.load(json_data)

# create a plain world map
world_map2 = folium.Map(location=sydney_coords, zoom_start=10, tiles='OpenStreetMap')

# generate choropleth map
folium.Choropleth(
    geo_data=data2,
    data=df_crime,
    columns=['region','crimes_2019'],
    key_on='feature.properties.nsw_lga__3',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crime rate in 2019',
    reset=True
).add_to(world_map2)

folium.LayerControl().add_to(world_map2)
# display map
world_map2

The city region of Sydney stands out as it has the highest crime rate (over 23%), followed by the outer western and southern regions (Penrith and Campbelltown). The crime rate seems to be lowest in the northern regions and around Sydney Harbour (excluding the city region), where it is below 6%.

## Conclusion
Based on the results there are a couple of recommendations to make for someone planning on moving to or staying in Sydney. 

Roughly, we can summarize neighborhoods in cluster 0 as having lots of fast food restaurant options and yoga studios (interesting combination...). For anyone looking for any or both of these venues then the western and outer regions would be suitable. If you love cafés, pubs or pizza places then a neighborhood in cluster 1, which is focused around the city center, seems to be the best choice. Over 80% of the suburbs here have café as their most common venue. If you are however looking for parks, yoga studios or egyptian restuarants then neighborhoods in cluster 3 seem to be good choices. These neighborhoods are spread out but can be found in various areas (see cluster map above).

Living expenses are affected by many factors but looking at median house values alone we can conclude that house values in coastal areas, eastern suburbs, and neighborhoods surrounding Chatswood and Gordon are higher than in areas further inland. One should expect to pay at least \$1,500,000 AUD for owning a house here. 

For someone concerned about crime, the rates from 2019 are highest in the city region and lowest in the northern regions. The crime rate is generally higher in the west than in the east. 